# Processing Dataset


### Functions



In [41]:
from datasets import * #suprisingly the only import needed.

In [42]:
# A dictionary of arabic letters from dotted to dotless. (credit: @kaddu341)
letter_dict = {
    "ا": "ا",
    "أ": "ا",
    "إ": "ا",
    "آ": "ا",
    "ب": "ب",
    "ت": "ب",
    "ث": "ب",
    "ج": "ح",
    "ح": "ح",
    "خ": "ح",
    "د": "د",
    "ذ": "د",
    "ر": "ر",
    "ز": "ر",
    "س": "س",
    "ش": "س",
    "ص": "ص",
    "ض": "ص",
    "ط": "ط",
    "ظ": "ط",
    "ع": "ع",
    "غ": "ع",
    "ف": "ف",
    "ق": "ف",
    "ك": "ك",
    "ل": "ل",
    "م": "م",
    "ن": "ن",
    "و": "و",
    "ؤ": "و",
    "ه": "ه",
    "ة": "ه",
    "ي": "ى",
    "ى": "ى",
    "ئ": "ى",
    " ": " ",
}
letter_set = set(letter_dict.keys())


In [43]:
# Replaces multiple spaces, or whatever char designated by the perameter into one char. (helper function)
def replace(string, char):
    while char+char in string:
        string = string.replace(char+char, char)

    return string


In [44]:
# Essential function, parses arabic dotted text into dotless text. 
# Returns a dict with keys "clean" and "dotless". 
# This format is required for the datasets.map() function.

#unused, see below functions, also this is not batched
def parse_text(text):
    
  text = text["text"].strip(" ")
  
  clean_chars = [char for char in text if char in letter_set]
  clean_string = ''.join(clean_chars)

  dotless_chars = [letter_dict[char] for char in clean_chars]
  dotless_string = ''.join(dotless_chars)
      
  clean_string = replace(clean_string, ' ')
  dotless_string = replace(dotless_string, '_')
  
  return {"clean": clean_string, "dotless": dotless_string}


In [45]:
def parse_clean_text(example):
  
  clean = []
  
  for text in example["text"]:
    text = text.strip(" ")
    
    clean_chars = [char for char in text if char in letter_set]
    clean_string = ''.join(clean_chars)
    
    clean_string = replace(clean_string, ' ')
    clean.append(clean_string)
  
  return {"clean": clean}

In [46]:
def parse_dotless_text(example):
  dotless = []
  for text in example["clean"]:
    dotless_chars = [letter_dict[char] for char in text]
    dotless_string = ''.join(dotless_chars)
        
    dotless_string = replace(dotless_string, '_')
    dotless.append(dotless_string)
  
  return {"dotless": dotless}

In [47]:
#test
text = {"text": "ولا تنسى التسجيل معنا لتستفيد بكل جديد"}
print(text, "\n")

text = parse_clean_text(text)
print(text, "\n")

text = parse_dotless_text(text)
print(text, "\n")


{'text': 'ولا تنسى التسجيل معنا لتستفيد بكل جديد'} 

{'clean': ['و', 'ل', 'ا', '', 'ت', 'ن', 'س', 'ى', '', 'ا', 'ل', 'ت', 'س', 'ج', 'ي', 'ل', '', 'م', 'ع', 'ن', 'ا', '', 'ل', 'ت', 'س', 'ت', 'ف', 'ي', 'د', '', 'ب', 'ك', 'ل', '', 'ج', 'د', 'ي', 'د']} 

{'dotless': ['و', 'ل', 'ا', '', 'ب', 'ن', 'س', 'ى', '', 'ا', 'ل', 'ب', 'س', 'ح', 'ى', 'ل', '', 'م', 'ع', 'ن', 'ا', '', 'ل', 'ب', 'س', 'ب', 'ف', 'ى', 'د', '', 'ب', 'ك', 'ل', '', 'ح', 'د', 'ى', 'د']} 



In [48]:
# Used for the datasets.filter() function to filter out examples/rows that have examples of less than 20 chars.
#def filterEmpty(x):
#    return not len(x['clean']) < 20

#Not used, lambda function used instead

### Loading, saving, and trimming Oscar (unshuffled_deduplicated_ar) dataset

In [49]:
#load the oscar unshuffled_deduplicated_ar dataset from huggingface 
#datasetLoad = load_dataset("oscar", "unshuffled_deduplicated_ar") #Oscar Arabic dataset

In [50]:
#save the dataset to "ar-arrow-datasets/" Apache Arrow datafolder
#datasetLoad.save_to_disk('ar-arrow-datasets')

In [51]:
#load from file
full_dataset = load_from_disk('ar-arrow-datasets')

In [52]:
print(full_dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'text'],
        num_rows: 9006977
    })
})


### Applying maps, filters, and splits
This process of applying maps and filters is very slow especially for big datasets. 

It is possible to implement a batched approach similar ot batched tokenization that would speed it up greatly. 

In the .map() add perameter batched=True. However the function called needs to be compatible with this approach.

I will implement this soon.

Generally this is not the way to handle big data. However this is fine for the AR-dotless-small dataset

In [53]:
# Gets the index of the #th space.
def num_word(text, num):
    space_count = 0
    index = -1
    for i in range(num):
        index = text.find(' ', index + 1)
    return index

In [54]:
import math
exampleToWordsLen = [] # just to keep track of how many chunks an old  made

In [55]:
print(exampleToWordsLen[:5])

[]


In [56]:

# Chunks examples into smaller examples, at @words_per_chuck words each
def chunk_examples(examples, words_per_chunk=8):
    chunks = []
    for sentence in examples["clean"]:
        words = sentence.split()
        num_words = len(words)
        exampleToWordsLen.append(math.ceil(num_words/words_per_chunk)) # just to keep track of how many chunks an old example made
        start = 0
        while start < num_words:
            end = min(start + words_per_chunk, num_words)
            chunk = ' '.join(words[start:end])
            chunks.append(chunk)
            start = end
    
    return {"clean": chunks}

In [57]:
dataset = full_dataset['train'].select(range(2400)) #Trim to number of examples
dataset = dataset.remove_columns(["id"])
print(dataset, '\n')

dataset = dataset.filter(lambda x: not len(x['text']) < 20) #Apply the filter to remove useless empty examples.
print(dataset, '\n')
        
dataset = dataset.map(parse_clean_text, batched=True, remove_columns="text") #Apply the function (convert dotted to dotless)
set_state_1 = dataset["clean"][0]
print(dataset, '\n')

# test to see how many examples are under 20 characters. Goal of filterEmpty() to eliminate
a = 0
for i in dataset['clean']:
    if len(i) < 40:
        a += 1
print(a, '\n')

dataset = dataset.map(chunk_examples, batched=True, remove_columns=dataset.column_names) # applies teh chuncking
set_state_2 = dataset["clean"][0:exampleToWordsLen[0]]
print(dataset, '\n')

dataset = dataset.map(parse_dotless_text, batched=True) #Apply the function (convert dotted to dotless)
set_state_3 = dataset["dotless"][0:exampleToWordsLen[0]]
print(dataset, '\n')

dataset = dataset.filter(lambda x: not len(x['clean']) < 20) #Apply the filter to remove useless empty examples.
print(dataset, '\n')

Dataset({
    features: ['text'],
    num_rows: 2400
}) 

Dataset({
    features: ['text'],
    num_rows: 2400
}) 

Dataset({
    features: ['clean'],
    num_rows: 2400
}) 

15 



Map: 100%|██████████| 2400/2400 [00:00<00:00, 23594.88 examples/s]

Dataset({
    features: ['clean'],
    num_rows: 104246
}) 




Map: 100%|██████████| 104246/104246 [00:00<00:00, 261817.97 examples/s]


Dataset({
    features: ['clean', 'dotless'],
    num_rows: 104246
}) 



Filter: 100%|██████████| 104246/104246 [00:00<00:00, 581243.71 examples/s]

Dataset({
    features: ['clean', 'dotless'],
    num_rows: 103404
}) 



In [58]:
print(set_state_1,"\n\n")
print(set_state_2,"\n\n")
print(set_state_3)

مرحبا بك عزيز الزائر نتمنى لك أوقاتا سعيدة معنا وأن نزداد شرفا بخدمتك ولا تنسى التسجيل معنا لتستفيد بكل جديدأهلا وسهلا بك زائرنا الكريم أنت لم تقم بتسجيل الدخول بعد يشرفنا أن تقوم بالدخول أو التسجيل إذا رغبت بالمشاركة في المنتدىنرحب بكل الزائرين ونتمى لكم قضا وقت ممتع معنا يملأه الحب والود والاستفادة المتبادلة بيننا علميا وعمليا يسعدنا تسجيلكم معنا ومشاركتنا وشعارنا دوما نحب الخير لكل الناس مهما اختلفت الألوان والديانات والأجناس لي أربع شقيقات أنا أكثرهن غنى لكن لا أدري لماذا يأتي أقاربي لزيارة أخواتي بكثرةوحينما يأتي موعد زيارتي لا يأتي سوى القليلفهم يزورون أخواتي الأربع كل يومأماأنا أكثر أخواتي عطا لمن يأتيني لا أتهم أخواتي بالتقصير أبدا ولكن الكل يعرف أني أكثرهن عطاكثيرون ينصحون أقاربي بأن يأتوني فلدي خير كثير وأعطي بكرم من يأتيني ومع ذلك يبتعدون عني فلا حياة لمن تنادي اختر منتدىقناة عجباوي التلفزيونية التسويقية البرامج التلفزيونية لقناة عجباوي التسويقية تعرف على عجباوي سيرة ذاتية سابقة أعمال أرشيف تصميمات عجباوي إعلانات الشركات والمحلات أغلفة كتب تنسيقات المتن الداخلي للكتب الدفاتر

In [59]:
# test to see how many examples are under 20 characters. Goal of filterEmpty() to eliminate
a = 0
for i in dataset['clean']:
    if len(i) < 20:
        a += 1
a

0

In [60]:
dataset_clean = dataset.train_test_split(train_size= 0.8) #create 80/20 split into train and test datasets
dataset_clean

DatasetDict({
    train: Dataset({
        features: ['clean', 'dotless'],
        num_rows: 82723
    })
    test: Dataset({
        features: ['clean', 'dotless'],
        num_rows: 20681
    })
})

### Save and upload to hugging face

In [61]:
#save the cleaned dataset to "clean-arrow-datasets/" Apache Arrow datafolder
dataset_clean.save_to_disk("clean-arrow-datasets")

Saving the dataset (1/1 shards): 100%|██████████| 20681/20681 [00:00<00:00, 514461.95 examples/s]


In [62]:
#required token id. Run 'huggingface-cli login'
dataset_clean.push_to_hub("dot-ammar/AR-dotless-small", private=True)

Deleting unused files from dataset repository: 100%|██████████| 1/1 [00:00<00:00,  7.81it/s]
